In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
google_drive_path = 'xxxxxxx'
os.chdir(google_drive_path)
print("Current working directory:", os.getcwd())
!ls

!python -m pip install trimesh
!python -m pip install rtree


Mounted at /content/gdrive
Current working directory: /content/gdrive/Othercomputers/My Mac/Physics-informed-shape-generation-master/Code
 Dataset_generation  'Function Validation and testing'	 Test_Cases
 File_Paths	      JEB_new_density			 Training_Scripts
 Functions	      Models
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 740.4/740.4 kB 32.9 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import plotly.graph_objects as go

def load_obj_tri_mesh(path):
    verts, faces = [], []
    with open(path, "r") as f:
        for line in f:
            if not line or line[0] in ("#", "o", "s", "g", "vt", "vn", "mtl", "usemtl"):
                continue
            if line.startswith("v "):
                _, x, y, z, *rest = line.strip().split()
                verts.append((float(x), float(y), float(z)))
            elif line.startswith("f "):
                parts = line.strip().split()[1:]
                if len(parts) != 3:
                    continue
                tri = []
                for token in parts:
                    tri.append(int(token.split('/')[0]) - 1)
                faces.append(tri)
    V = np.asarray(verts, dtype=np.float32)
    F = np.asarray(faces, dtype=np.int32)
    return V, F

def decimate_faces_for_preview(verts, faces, max_faces=900_000, seed=0):
    if faces.shape[0] <= max_faces:
        return verts, faces
    rng = np.random.default_rng(seed)
    sel = rng.choice(faces.shape[0], size=max_faces, replace=False)
    Fsel = faces[sel]
    uniq, inv = np.unique(Fsel, return_inverse=True)
    Vnew = verts[uniq]
    Fnew = inv.reshape(-1, 3)
    return Vnew, Fnew

path = "xxxxx/iso-000010.obj"
V, F = load_obj_tri_mesh(path)
V, F = decimate_faces_for_preview(V, F, max_faces=700_000)


fig = go.Figure(
    data=[go.Mesh3d(
        x=V[:, 0], y=V[:, 1], z=V[:, 2],
        i=F[:, 0], j=F[:, 1], k=F[:, 2],
        color="rgb(0, 102, 255)",
        opacity=1.0,
        flatshading=False,
        lighting=dict(
            ambient=0.45,
            diffuse=0.65,
            specular=0.25,
            roughness=0.35
        ),
        lightposition=dict(x=1.5, y=0.0, z=2.0),
        name="closed iso"
    )]
)

fig.update_layout(
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False),
        bgcolor="white",
        aspectmode="data"
    ),
    paper_bgcolor="white",
    margin=dict(l=0, r=0, b=0, t=0),
    showlegend=False
)

fig.show()


In [ ]:
import trimesh
obj = " "
mesh = trimesh.load(obj)
mesh = trimesh.smoothing.filter_laplacian(mesh, lamb=0.2, iterations=1)
mesh.show()

ValueError: string is not a file: ` `